In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

In [3]:
# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


In [4]:
new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11','form_field41','form_field40', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field41','form_field40', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])

for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

In [15]:
test_given_new = test_given[test_new_features]
test_given_new.head()

,form_field1,form_field2,form_field3,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field12,...,form_field37,form_field38,form_field42,form_field43,form_field44,form_field46,form_field47,form_field48,form_field49,form_field50
0,3236.0,0.34875,10.2006,0.0,418564.0,418564.0,418564.0,540710.0,0.0,5446.0,...,2.0,14.0,0.825000,1.01,0.800000,0.0,1,-999.000000,0.000000,0.011221
1,3284.0,1.27360,2.9606,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,1025793.0,...,18.0,14.0,0.507694,4.04,0.623248,0.0,0,-999.000000,0.504974,0.043525
2,-999.0,0.27505,0.0600,0.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,-999.0,0.0,-999.000000,0.00,-999.000000,-999.0,1,-999.000000,0.000000,-999.000000
3,3232.0,0.28505,2.8032,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1176725.0,...,0.0,26.0,0.916663,2.02,0.464224,-999.0,1,90.163742,0.788809,0.104029
4,3466.0,2.09545,0.8318,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,613064.0,...,28.0,20.0,0.234047,23.23,0.726688,0.0,0,1303.587148,1.637733,0.163124


In [5]:
X = train[new_features]
Y = train['default_status']

In [8]:
from sklearn.model_selection import KFold,StratifiedKFold
from catboost import CatBoostClassifier

#{'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
 #         ,'early_stopping_rounds':200,'use_best_model':True}

params = {'n_estimators':3500,'depth':12,'learning_rate':0.05,
          'l2_leaf_reg':10, 'border_count':100,'thread_count':4,
          'random_seed': 1234, 'objective':'CrossEntropy',
          'eval_metric':'AUC','early_stopping_rounds':200,'use_best_model':True}

In [16]:
from sklearn.model_selection import KFold,StratifiedKFold
from catboost import CatBoostClassifier

fold=StratifiedKFold(n_splits=15) #5 # 10 
i=1
for train_index, test_index in fold.split(X,Y):
    
    #The usual cross validation
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    model  = CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_train,Y_train),(X_test, Y_test)], early_stopping_rounds=300,verbose=100,)#erly100
    preds = model.predict(X_test)
    
    
    #Simultaneosly Predicting on test set for submission
    pred = model.predict_proba(test_given_new)[:,1]

0:	test: 0.8142578	test1: 0.8002725	best: 0.8002725 (0)	total: 271ms	remaining: 15m 49s
100:	test: 0.8958531	test1: 0.8252180	best: 0.8254208 (90)	total: 1m 5s	remaining: 36m 31s
200:	test: 0.9321093	test1: 0.8272922	best: 0.8273302 (179)	total: 2m 13s	remaining: 36m 30s
300:	test: 0.9582063	test1: 0.8264784	best: 0.8277454 (261)	total: 3m 15s	remaining: 34m 36s
400:	test: 0.9729994	test1: 0.8243527	best: 0.8277454 (261)	total: 4m 18s	remaining: 33m 15s
500:	test: 0.9839229	test1: 0.8235548	best: 0.8277454 (261)	total: 5m 22s	remaining: 32m 10s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8277453734
bestIteration = 261

Shrink model to first 262 iterations.
0:	test: 0.8264190	test1: 0.8151588	best: 0.8151588 (0)	total: 995ms	remaining: 58m 3s
100:	test: 0.8953692	test1: 0.8437469	best: 0.8437469 (100)	total: 1m 8s	remaining: 38m 29s
200:	test: 0.9330999	test1: 0.8473508	best: 0.8475497 (194)	total: 2m 14s	remaining: 36m 44s
300:	test: 0.9604821	test1: 0.8481708	

400:	test: 0.9753792	test1: 0.8568234	best: 0.8630957 (102)	total: 4m 33s	remaining: 35m 15s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8630956863
bestIteration = 102

Shrink model to first 103 iterations.
0:	test: 0.8273855	test1: 0.7954896	best: 0.7954896 (0)	total: 710ms	remaining: 41m 25s
100:	test: 0.8967731	test1: 0.8314342	best: 0.8314342 (100)	total: 1m 12s	remaining: 40m 47s
200:	test: 0.9349044	test1: 0.8332692	best: 0.8332692 (200)	total: 2m 20s	remaining: 38m 27s
300:	test: 0.9610608	test1: 0.8335335	best: 0.8337656 (277)	total: 3m 31s	remaining: 37m 24s
400:	test: 0.9766439	test1: 0.8318071	best: 0.8337656 (277)	total: 4m 45s	remaining: 36m 49s
500:	test: 0.9849639	test1: 0.8301740	best: 0.8337656 (277)	total: 6m 7s	remaining: 36m 40s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8337655624
bestIteration = 277

Shrink model to first 278 iterations.
0:	test: 0.8129910	test1: 0.8189487	best: 0.8189487 (0)	total: 255ms	remainin

In [40]:
#pred_proba = pred[:,1]

In [17]:
#file for submission
id = test_given['Applicant_ID']
#id

sample_submission = pd.DataFrame(id, columns=['Applicant_ID'])
sample_submission['default_status'] = pred
sample_submission

sample_submission.to_csv('My_submissionPer1.csv', index=False)

In [20]:
corr_matrix = train.corr()
corr_matrix['default_status'].sort_values(ascending = False)
#sns.heatmap(corr_matrix['default_status'].sort_values(ascending = False))

default_status    1.000000
form_field23      0.318622
form_field17      0.269507
form_field18      0.265692
form_field35      0.254479
form_field47      0.227926
form_field40      0.164199
form_field45      0.154638
form_field4       0.147548
form_field39      0.145949
form_field22      0.134407
form_field41      0.118484
form_field2       0.073846
form_field36      0.040382
form_field11      0.033700
form_field19      0.026958
form_field38      0.019113
form_field5       0.015713
form_field3       0.009359
form_field14      0.008272
form_field49      0.007601
form_field20      0.007007
form_field34      0.005449
form_field42     -0.009512
form_field43     -0.012350
form_field33     -0.032960
form_field50     -0.048947
form_field48     -0.055441
form_field46     -0.073562
form_field12     -0.088259
form_field13     -0.091342
form_field44     -0.112267
form_field15     -0.117442
form_field31     -0.118339
form_field7      -0.120380
form_field24     -0.128646
form_field30     -0.132381
f

In [21]:
train.T.drop_duplicates()

,0,1,2,3,4,5,6,7,8,9,...,55990,55991,55992,55993,55994,55995,55996,55997,55998,55999
Applicant_ID,Apcnt_1000000,Apcnt_1000004,Apcnt_1000008,Apcnt_1000012,Apcnt_1000016,Apcnt_1000020,Apcnt_1000024,Apcnt_1000028,Apcnt_1000036,Apcnt_1000040,...,Apcnt_999944,Apcnt_999948,Apcnt_999952,Apcnt_999960,Apcnt_999964,Apcnt_999968,Apcnt_999972,Apcnt_999980,Apcnt_999988,Apcnt_999996
form_field1,3436,3456,3276,3372,3370,3724,3350,3416,3376,3310,...,3600,-999,3504,3556,-999,3740,3360,3500,3280,3522
form_field2,0.28505,0.674,0.53845,0.17005,0.7727,-999,0.24205,0.8702,0.2918,0.24395,...,0.50335,0.2267,0.39085,0.6934,0.01465,0.0173,2.01145,0.7664,0.05235,0.4693
form_field3,1.656,0.2342,3.151,0.505,1.101,0,0.3132,0.057,0.5586,0.1562,...,0,0,0.098,0.0736,0,0,0.6252,0,2.0916,0
form_field4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.2212,0
form_field5,0,0,6.282,0,0,0,2.8188,0,0,0,...,0,0,0,0,0,0,0,0,0,0
form_field6,0,0,-999,192166,1556,192944,-999,17505,18283,-999,...,755049,-999,4279,96472,-999,770998,-999,118645,-999,98806
form_field7,1.06897e+07,898979,956940,3.0447e+06,214728,9.24458e+06,580388,186331,1.10515e+06,0,...,1.20061e+07,-999,500254,3.46949e+06,-999,9.63748e+06,927765,3.66244e+06,3.4586e+06,2.05392e+06
form_field8,252072,497531,-999,385499,214728,6.76121e+06,-999,186331,18283,-999,...,1.9275e+06,-999,80912,3.46949e+06,-999,4.04793e+06,-999,3.66244e+06,-999,523983
form_field9,4.27278e+06,9.07381e+06,192944,3.98647e+06,1.28409e+06,6.79634e+07,96472,1.48481e+06,115533,-999,...,4.01397e+07,-999,2.81947e+06,2.82912e+07,-999,1.1642e+07,-999,3.58502e+06,115533,1.49034e+07


In [22]:
train.isnull().sum()

Applicant_ID          0
form_field1           0
form_field2           0
form_field3           0
form_field4         355
form_field5           0
form_field6           0
form_field7           0
form_field8           0
form_field9           0
form_field10          0
form_field11      31421
form_field12          0
form_field13          0
form_field14          0
form_field15          0
form_field16          0
form_field17      11151
form_field18      10402
form_field19          0
form_field20          0
form_field21          0
form_field22          0
form_field23      28123
form_field24          0
form_field25          0
form_field26          0
form_field27          0
form_field28          0
form_field29          0
form_field30      25509
form_field31      39408
form_field32          0
form_field33          0
form_field34          0
form_field35      23148
form_field36          0
form_field37          0
form_field38          0
form_field39       4211
form_field40      43729
form_field41    

In [3]:
'''
# Blending


Comp_submission_light-999stra10:0.840263
My_submissionPer:0.8417805
My_submissionlearn2_catboost-999stra15:0.8418536
My_submission_0.015catboost-999stra154200:0.841938
My_submission_catboost-999stra10:0.8417057
My_submission_catboost-999stra15:0.842020557
My_submission_catboost-999stra20:0.841973
DSN_Hack_ear_cat-999stra15:0.8417863

0.8418536 0.841938 0.842020557 0.841973
'''


'\n# Blending\n\n\nComp_submission_light-999stra10:0.840263\nMy_submissionPer:0.8417805\nMy_submissionlearn2_catboost-999stra15:0.8418536\nMy_submission_0.015catboost-999stra154200:0.841938\nMy_submission_catboost-999stra10:0.8417057\nMy_submission_catboost-999stra15:0.842020557\nMy_submission_catboost-999stra20:0.841973\nDSN_Hack_ear_cat-999stra15:0.8417863\nDSN_Hack_New_cat-999stra15:0.8416199    \nDSN_Hack_cat-999stra5:0.8408600\n'

In [8]:
pred1 = pd.read_csv('C:/Users/USER/Comp_submission(light-999stra10).csv')['default_status']
pred2 = pd.read_csv('C:/Users/USER/My_submissionPer.csv')['default_status']
pred3 = pd.read_csv('C:/Users/USER/My_submissionlearn2(catboost-999stra15).csv')['default_status']
pred4 = pd.read_csv('C:/Users/USER/My_submission(0.015catboost-999stra154200).csv')['default_status']
pred5 = pd.read_csv('C:/Users/USER/My_submission(catboost-999stra10).csv')['default_status']
pred6 = pd.read_csv('C:/Users/USER/My_submission(catboost-999stra15).csv')['default_status']
pred7 = pd.read_csv('C:/Users/USER/My_submission(catboost-999stra20).csv')['default_status']
pred8 = pd.read_csv('C:/Users/USER/DSN_Hack_ear(cat-999stra15).csv')['default_status']

In [13]:
sub_pred = (pred3*0.15) + (pred4*0.20) + (pred6*0.65)

In [14]:
sample_submission['default_status'] = sub_pred

In [16]:
sample_submission.to_csv('Blend.csv', index = False)

In [21]:
sub_pred2 = (pred4*0.2) + (pred6*0.45)+ (pred7*0.35)

In [22]:
sample_submission['default_status'] = sub_pred2

In [23]:
sample_submission.to_csv('Blend2.csv', index = False)

In [26]:
sub_pred3 = (pred3*0.025) + (pred4*0.075) + (pred6*0.55) + (pred7*0.35)
sample_submission['default_status'] = sub_pred3
sample_submission.to_csv('Blend3.csv', index = False)

In [27]:
sub_pred4 =  (pred6*0.5) + (pred7*0.5)
sample_submission['default_status'] = sub_pred4
sample_submission.to_csv('Blend4.csv', index = False)

In [28]:
sub_pred5 = (pred3*0.20) + (pred4*0.25) + (pred6*0.55)
sample_submission['default_status'] = sub_pred5
sample_submission.to_csv('Blend5.csv', index = False)

In [29]:
# BEST BLEND
sub_pred6 = (pred3*0.20) + (pred4*0.25) + (pred6*0.45)+ (pred5*0.1)
sample_submission['default_status'] = sub_pred6
sample_submission.to_csv('Blend6.csv', index = False)

In [30]:
sub_pred7 = (pred3*0.15) + (pred4*0.2) + (pred6*0.5)+(pred5*0.05) + (pred7*0.05) + (pred2*0.05)
sample_submission['default_status'] = sub_pred7
sample_submission.to_csv('Blend7.csv', index = False)

# Blending max depths

In [3]:
pred1 = pd.read_csv('C:/Users/USER/Submission_new2.csv')['default_status']
pred2 = pd.read_csv('C:/Users/USER/Sub_new.csv')['default_status']
pred3 = pd.read_csv('C:/Users/USER/Submission_new3.csv')['default_status']

In [6]:
sub_pred_5 = (pred3*0.65) + (pred1*0.25) + (pred2*0.1)
sample_submission['default_status'] = sub_pred_5
sample_submission.to_csv('Blend_new.csv', index = False)

In [7]:
sub_pred_6 = (pred3*0.45) + (pred1*0.35) + (pred2*0.2)
sample_submission['default_status'] = sub_pred_6
sample_submission.to_csv('Blend_new1.csv', index = False)

In [9]:
pred1 = pd.read_csv('C:/Users/USER/Submission_new2.csv')['default_status']
pred2 = pd.read_csv('C:/Users/USER/Sub_new.csv')['default_status']
pred3 = pd.read_csv('C:/Users/USER/Submision_new4.csv')['default_status']

In [10]:
sub_pred_7 = (pred3*0.65) + (pred1*0.25) + (pred2*0.1)
sample_submission['default_status'] = sub_pred_7
sample_submission.to_csv('Blend_new2.csv', index = False)